# Hodina 13: Informované prohledávání (A*, heuristiky) 🎯

## Přehled lekce

V této hodině se naučíme:
- Implementovat A* algoritmus s neuronovými heuristikami
- Trénovat transformery pro učení heuristik
- Vizualizovat informované prohledávání
- Porovnat různé heuristiky a jejich efektivitu
- Vytvořit interaktivní aplikace pro exploraci A*

---

## 1. Nastavení prostředí a instalace knihoven

In [ ]:
# Instalace potřebných knihoven
!pip install torch torchvision transformers gradio networkx matplotlib numpy
!pip install plotly ipywidgets pygame celluloid scikit-learn

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.patches import Rectangle
import networkx as nx
from collections import defaultdict
import heapq
import time
from IPython.display import HTML, display, clear_output
import plotly.graph_objects as go
import gradio as gr
from transformers import GPT2Model, GPT2Config
import json
from celluloid import Camera
import random
from typing import List, Tuple, Dict, Set

# Nastavení zobrazení
plt.style.use('seaborn-v0_8-darkgrid')
np.random.seed(42)
torch.manual_seed(42)

## 2. A* Algoritmus - Teorie a implementace ⭐

A* kombinuje skutečnou vzdálenost od startu (g) s odhadem vzdálenosti k cíli (h).

In [ ]:
# Implementace A* s vizualizací
class AStarVisualizer:
    def __init__(self):
        self.open_set = []
        self.closed_set = set()
        self.g_score = {}
        self.f_score = {}
        self.parent = {}
        self.search_history = []
    
    def heuristic(self, node, goal, heuristic_type='manhattan'):
        """Různé typy heuristik"""
        if heuristic_type == 'manhattan':
            return abs(node[0] - goal[0]) + abs(node[1] - goal[1])
        elif heuristic_type == 'euclidean':
            return np.sqrt((node[0] - goal[0])**2 + (node[1] - goal[1])**2)
        elif heuristic_type == 'zero':
            return 0  # Degeneruje na Dijkstru
        else:
            return 0
    
    def search(self, graph, start, goal, heuristic_func=None):
        """A* algoritmus s ukládáním kroků"""
        if heuristic_func is None:
            heuristic_func = lambda n: self.heuristic(n, goal)
        
        # Inicializace
        self.g_score[start] = 0
        self.f_score[start] = heuristic_func(start)
        heapq.heappush(self.open_set, (self.f_score[start], start))
        
        while self.open_set:
            current_f, current = heapq.heappop(self.open_set)
            
            # Uložení kroku
            self.search_history.append({
                'current': current,
                'open_set': [node for _, node in self.open_set],
                'closed_set': set(self.closed_set),
                'g_scores': dict(self.g_score),
                'f_scores': dict(self.f_score)
            })
            
            if current == goal:
                return self._reconstruct_path(current)
            
            self.closed_set.add(current)
            
            for neighbor, weight in graph[current]:
                if neighbor in self.closed_set:
                    continue
                
                tentative_g = self.g_score[current] + weight
                
                if neighbor not in self.g_score or tentative_g < self.g_score[neighbor]:
                    self.parent[neighbor] = current
                    self.g_score[neighbor] = tentative_g
                    self.f_score[neighbor] = tentative_g + heuristic_func(neighbor)
                    
                    if neighbor not in [n for _, n in self.open_set]:
                        heapq.heappush(self.open_set, (self.f_score[neighbor], neighbor))
        
        return None
    
    def _reconstruct_path(self, current):
        path = [current]
        while current in self.parent:
            current = self.parent[current]
            path.append(current)
        return path[::-1]

# Vytvoření mřížkového světa pro demonstraci
class GridWorld:
    def __init__(self, width=20, height=20, obstacles_ratio=0.2):
        self.width = width
        self.height = height
        self.grid = np.zeros((height, width))
        self._add_obstacles(obstacles_ratio)
    
    def _add_obstacles(self, ratio):
        n_obstacles = int(self.width * self.height * ratio)
        for _ in range(n_obstacles):
            x = np.random.randint(0, self.width)
            y = np.random.randint(0, self.height)
            if (x, y) not in [(0, 0), (self.width-1, self.height-1)]:
                self.grid[y, x] = 1
    
    def get_neighbors(self, pos):
        x, y = pos
        neighbors = []
        
        # 8 směrů pohybu
        for dx in [-1, 0, 1]:
            for dy in [-1, 0, 1]:
                if dx == 0 and dy == 0:
                    continue
                
                nx, ny = x + dx, y + dy
                if (0 <= nx < self.width and 0 <= ny < self.height and 
                    self.grid[ny, nx] == 0):
                    # Diagonální pohyby mají vyšší cenu
                    cost = 1.414 if dx != 0 and dy != 0 else 1.0
                    neighbors.append(((nx, ny), cost))
        
        return neighbors
    
    def to_graph(self):
        """Převede mřížku na graf pro A*"""
        graph = defaultdict(list)
        for y in range(self.height):
            for x in range(self.width):
                if self.grid[y, x] == 0:
                    graph[(x, y)] = self.get_neighbors((x, y))
        return graph
    
    def visualize(self, path=None, search_history=None, step=-1):
        """Vizualizuje mřížku s cestou a historií prohledávání"""
        fig, ax = plt.subplots(figsize=(10, 10))
        
        # Základní mřížka
        ax.imshow(self.grid, cmap='binary', origin='lower')
        
        # Historie prohledávání
        if search_history and 0 <= step < len(search_history):
            history = search_history[step]
            
            # Closed set (prozkoumané)
            for node in history['closed_set']:
                ax.add_patch(Rectangle((node[0]-0.5, node[1]-0.5), 1, 1, 
                                     facecolor='lightblue', alpha=0.5))
            
            # Open set (k prozkoumání)
            for node in history['open_set']:
                ax.add_patch(Rectangle((node[0]-0.5, node[1]-0.5), 1, 1, 
                                     facecolor='yellow', alpha=0.5))
            
            # Aktuální uzel
            current = history['current']
            ax.add_patch(Rectangle((current[0]-0.5, current[1]-0.5), 1, 1, 
                                 facecolor='red', alpha=0.7))
            
            # F-score hodnoty
            for node, f_score in history['f_scores'].items():
                ax.text(node[0], node[1], f'{f_score:.1f}', 
                       ha='center', va='center', fontsize=8)
        
        # Finální cesta
        if path:
            path_x = [p[0] for p in path]
            path_y = [p[1] for p in path]
            ax.plot(path_x, path_y, 'g-', linewidth=3, alpha=0.7)
        
        # Start a cíl
        ax.plot(0, 0, 'go', markersize=15, label='Start')
        ax.plot(self.width-1, self.height-1, 'ro', markersize=15, label='Cíl')
        
        ax.set_xlim(-0.5, self.width-0.5)
        ax.set_ylim(-0.5, self.height-0.5)
        ax.set_aspect('equal')
        ax.grid(True, alpha=0.3)
        ax.legend()
        ax.set_title(f'A* Prohledávání - Krok {step+1}' if step >= 0 else 'A* Výsledek')
        
        return fig

# Demonstrace A*
print("🌟 Demonstrace A* algoritmu:")
world = GridWorld(15, 15, 0.15)
graph = world.to_graph()

# Spuštění A*
astar = AStarVisualizer()
start = (0, 0)
goal = (14, 14)
path = astar.search(graph, start, goal)

print(f"Cesta nalezena! Délka: {len(path)} kroků")
print(f"Počet prozkoumaných uzlů: {len(astar.closed_set)}")

# Vizualizace výsledku
fig = world.visualize(path=path)
plt.show()

## 3. Neuronové heuristiky pro A* 🧠

Naučíme neuronovou síť předpovídat optimální heuristiky.

In [ ]:
# Neuronová síť pro učení heuristik
class NeuralHeuristic(nn.Module):
    def __init__(self, grid_size=15, hidden_dim=256):
        super(NeuralHeuristic, self).__init__()
        
        # CNN pro extrakci příznaků z mřížky
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((4, 4))
        )
        
        # Plně propojené vrstvy
        self.fc_layers = nn.Sequential(
            nn.Linear(128 * 4 * 4 + 4, hidden_dim),  # +4 pro pozice
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Linear(hidden_dim // 2, 1)
        )
    
    def forward(self, grid, current_pos, goal_pos):
        # grid: [batch, 3, H, W] - 3 kanály: překážky, current, goal
        conv_features = self.conv_layers(grid)
        conv_features = conv_features.view(conv_features.size(0), -1)
        
        # Spojení s pozicemi
        positions = torch.cat([current_pos, goal_pos], dim=1)
        features = torch.cat([conv_features, positions], dim=1)
        
        # Predikce vzdálenosti
        distance = self.fc_layers(features)
        return distance

# Generování trénovacích dat
def generate_training_data(n_samples=1000, grid_size=15):
    """Generuje trénovací data pro neuronovou heuristiku"""
    X_grids = []
    X_current = []
    X_goal = []
    y_distances = []
    
    for _ in range(n_samples):
        # Vytvoření náhodné mřížky
        world = GridWorld(grid_size, grid_size, 0.15)
        graph = world.to_graph()
        
        # Náhodné pozice
        valid_positions = [(x, y) for x in range(grid_size) 
                          for y in range(grid_size) 
                          if world.grid[y, x] == 0]
        
        if len(valid_positions) < 2:
            continue
        
        start = random.choice(valid_positions)
        goal = random.choice([p for p in valid_positions if p != start])
        
        # Spuštění A* pro získání skutečné vzdálenosti
        astar = AStarVisualizer()
        path = astar.search(graph, start, goal)
        
        if path:
            # Vytvoření 3-kanálového vstupu
            grid_input = np.zeros((3, grid_size, grid_size))
            grid_input[0] = world.grid  # Překážky
            grid_input[1, start[1], start[0]] = 1  # Start
            grid_input[2, goal[1], goal[0]] = 1  # Cíl
            
            X_grids.append(grid_input)
            X_current.append(start)
            X_goal.append(goal)
            y_distances.append(len(path) - 1)
    
    return (torch.FloatTensor(X_grids), 
            torch.FloatTensor(X_current),
            torch.FloatTensor(X_goal),
            torch.FloatTensor(y_distances))

# Trénování neuronové heuristiky
print("🎯 Generování trénovacích dat...")
X_grids, X_current, X_goal, y_distances = generate_training_data(500, 15)

print(f"Vygenerováno {len(X_grids)} vzorků")

# Vytvoření a trénování modelu
model = NeuralHeuristic(grid_size=15)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

print("\n🏋️ Trénování neuronové heuristiky...")
losses = []

for epoch in range(100):
    optimizer.zero_grad()
    
    predictions = model(X_grids, X_current, X_goal)
    loss = criterion(predictions.squeeze(), y_distances)
    
    loss.backward()
    optimizer.step()
    
    losses.append(loss.item())
    
    if epoch % 20 == 0:
        print(f"Epocha {epoch}: Loss = {loss.item():.4f}")

# Vizualizace trénování
plt.figure(figsize=(10, 5))
plt.plot(losses)
plt.title('Průběh trénování neuronové heuristiky')
plt.xlabel('Epocha')
plt.ylabel('MSE Loss')
plt.grid(True)
plt.show()

print("\n✅ Model natrénován!")

## 4. Porovnání různých heuristik 📊

Porovnáme klasické heuristiky s naší neuronovou heuristikou.

In [ ]:
# Porovnání heuristik
class HeuristicComparator:
    def __init__(self, neural_model=None):
        self.neural_model = neural_model
        self.results = {}
    
    def manhattan_heuristic(self, pos, goal):
        return abs(pos[0] - goal[0]) + abs(pos[1] - goal[1])
    
    def euclidean_heuristic(self, pos, goal):
        return np.sqrt((pos[0] - goal[0])**2 + (pos[1] - goal[1])**2)
    
    def neural_heuristic(self, world, pos, goal):
        if self.neural_model is None:
            return 0
        
        # Příprava vstupu
        grid_input = torch.zeros(1, 3, world.height, world.width)
        grid_input[0, 0] = torch.FloatTensor(world.grid)
        grid_input[0, 1, pos[1], pos[0]] = 1
        grid_input[0, 2, goal[1], goal[0]] = 1
        
        current_tensor = torch.FloatTensor([[pos[0], pos[1]]])
        goal_tensor = torch.FloatTensor([[goal[0], goal[1]]])
        
        with torch.no_grad():
            prediction = self.neural_model(grid_input, current_tensor, goal_tensor)
        
        return prediction.item()
    
    def compare_on_problem(self, world, start, goal):
        """Porovná všechny heuristiky na jednom problému"""
        graph = world.to_graph()
        results = {}
        
        # Manhattan heuristika
        astar_manhattan = AStarVisualizer()
        path_manhattan = astar_manhattan.search(
            graph, start, goal, 
            lambda n: self.manhattan_heuristic(n, goal)
        )
        results['manhattan'] = {
            'path_length': len(path_manhattan) if path_manhattan else np.inf,
            'nodes_explored': len(astar_manhattan.closed_set),
            'path': path_manhattan
        }
        
        # Euklidovská heuristika
        astar_euclidean = AStarVisualizer()
        path_euclidean = astar_euclidean.search(
            graph, start, goal,
            lambda n: self.euclidean_heuristic(n, goal)
        )
        results['euclidean'] = {
            'path_length': len(path_euclidean) if path_euclidean else np.inf,
            'nodes_explored': len(astar_euclidean.closed_set),
            'path': path_euclidean
        }
        
        # Neuronová heuristika
        if self.neural_model:
            astar_neural = AStarVisualizer()
            path_neural = astar_neural.search(
                graph, start, goal,
                lambda n: self.neural_heuristic(world, n, goal)
            )
            results['neural'] = {
                'path_length': len(path_neural) if path_neural else np.inf,
                'nodes_explored': len(astar_neural.closed_set),
                'path': path_neural
            }
        
        # Dijkstra (nulová heuristika)
        astar_dijkstra = AStarVisualizer()
        path_dijkstra = astar_dijkstra.search(
            graph, start, goal,
            lambda n: 0
        )
        results['dijkstra'] = {
            'path_length': len(path_dijkstra) if path_dijkstra else np.inf,
            'nodes_explored': len(astar_dijkstra.closed_set),
            'path': path_dijkstra
        }
        
        return results
    
    def visualize_comparison(self, results, world):
        """Vizualizuje porovnání heuristik"""
        fig, axes = plt.subplots(2, 2, figsize=(12, 12))
        axes = axes.flatten()
        
        heuristics = ['manhattan', 'euclidean', 'neural', 'dijkstra']
        colors = ['red', 'blue', 'green', 'orange']
        
        for idx, (heuristic, color) in enumerate(zip(heuristics, colors)):
            if heuristic not in results:
                continue
            
            ax = axes[idx]
            ax.imshow(world.grid, cmap='binary', origin='lower')
            
            # Cesta
            path = results[heuristic]['path']
            if path:
                path_x = [p[0] for p in path]
                path_y = [p[1] for p in path]
                ax.plot(path_x, path_y, color=color, linewidth=3, alpha=0.7)
            
            # Start a cíl
            ax.plot(0, 0, 'go', markersize=10)
            ax.plot(world.width-1, world.height-1, 'ro', markersize=10)
            
            # Statistiky
            nodes = results[heuristic]['nodes_explored']
            length = results[heuristic]['path_length']
            ax.set_title(f'{heuristic.capitalize()}\nUzlů: {nodes}, Délka: {length}')
            ax.grid(True, alpha=0.3)
        
        plt.tight_layout()
        return fig

# Spuštění porovnání
print("📊 Porovnání heuristik:")
comparator = HeuristicComparator(neural_model=model)

# Vytvoření testovacího problému
test_world = GridWorld(20, 20, 0.2)
start = (0, 0)
goal = (19, 19)

# Porovnání
results = comparator.compare_on_problem(test_world, start, goal)

# Výsledky
print("\nVýsledky:")
for heuristic, data in results.items():
    print(f"{heuristic.capitalize()}:")
    print(f"  - Prozkoumaných uzlů: {data['nodes_explored']}")
    print(f"  - Délka cesty: {data['path_length']}")

# Vizualizace
fig = comparator.visualize_comparison(results, test_world)
plt.show()

## 5. Transformer pro plánování cest 🤖

Použijeme transformer architekturu pro end-to-end plánování.

In [ ]:
# Transformer pro plánování cest
class TransformerPathPlanner(nn.Module):
    def __init__(self, grid_size=20, d_model=256, n_heads=8, n_layers=6):
        super(TransformerPathPlanner, self).__init__()
        self.grid_size = grid_size
        self.d_model = d_model
        
        # Pozicový encoding
        self.pos_encoding = self._create_positional_encoding(grid_size * grid_size, d_model)
        
        # Embedding pro různé typy buněk
        self.cell_embedding = nn.Embedding(5, d_model)  # prázdná, zeď, start, cíl, cesta
        
        # Transformer encoder
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=n_heads,
            dim_feedforward=d_model * 4,
            dropout=0.1,
            batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=n_layers)
        
        # Dekodér pro predikci dalšího kroku
        self.decoder = nn.Sequential(
            nn.Linear(d_model, d_model // 2),
            nn.ReLU(),
            nn.Linear(d_model // 2, grid_size * grid_size)
        )
    
    def _create_positional_encoding(self, max_len, d_model):
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1).float()
        
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * 
                           -(np.log(10000.0) / d_model))
        
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        
        return pe.unsqueeze(0)
    
    def forward(self, grid_sequence):
        # grid_sequence: [batch, seq_len, grid_size, grid_size]
        batch_size = grid_sequence.size(0)
        seq_len = grid_sequence.size(1)
        
        # Flatten grid a embed
        grid_flat = grid_sequence.view(batch_size, seq_len, -1)
        embedded = self.cell_embedding(grid_flat.long())
        
        # Přidání pozičního encodingu
        embedded += self.pos_encoding[:, :seq_len]
        
        # Transformer processing
        encoded = self.transformer(embedded[:, -1, :].unsqueeze(1))  # Pouze poslední stav
        
        # Predikce dalšího kroku
        next_step_logits = self.decoder(encoded.squeeze(1))
        
        return next_step_logits.view(batch_size, self.grid_size, self.grid_size)

# Prostředí pro učení transformeru
class PathPlanningEnvironment:
    def __init__(self, grid_size=20):
        self.grid_size = grid_size
        self.reset()
    
    def reset(self):
        # Vytvoření nové mřížky
        self.world = GridWorld(self.grid_size, self.grid_size, 0.15)
        
        # Náhodný start a cíl
        valid_positions = [(x, y) for x in range(self.grid_size) 
                          for y in range(self.grid_size) 
                          if self.world.grid[y, x] == 0]
        
        self.start = random.choice(valid_positions)
        self.goal = random.choice([p for p in valid_positions if p != self.start])
        
        # Aktuální pozice
        self.current_pos = self.start
        self.path = [self.start]
        
        return self.get_state()
    
    def get_state(self):
        """Vrátí aktuální stav jako grid s různými hodnotami"""
        state = self.world.grid.copy()
        state[self.start[1], self.start[0]] = 2  # Start
        state[self.goal[1], self.goal[0]] = 3    # Cíl
        
        # Cesta
        for pos in self.path[1:-1]:
            state[pos[1], pos[0]] = 4
        
        if self.current_pos != self.start and self.current_pos != self.goal:
            state[self.current_pos[1], self.current_pos[0]] = 4
        
        return state
    
    def step(self, action):
        """Provede krok podle akce (pozice na mřížce)"""
        x, y = action % self.grid_size, action // self.grid_size
        
        # Kontrola validity
        if (abs(x - self.current_pos[0]) <= 1 and 
            abs(y - self.current_pos[1]) <= 1 and
            self.world.grid[y, x] == 0):
            
            self.current_pos = (x, y)
            self.path.append(self.current_pos)
            
            if self.current_pos == self.goal:
                reward = 100
                done = True
            else:
                reward = -1
                done = False
        else:
            reward = -10  # Penalizace za neplatný tah
            done = False
        
        return self.get_state(), reward, done

# Demonstrace
print("🤖 Transformer pro plánování cest:")
planner = TransformerPathPlanner(grid_size=20)
print(f"Model má {sum(p.numel() for p in planner.parameters())} parametrů")

# Test inference
env = PathPlanningEnvironment()
state = env.reset()
state_tensor = torch.FloatTensor(state).unsqueeze(0).unsqueeze(0)

with torch.no_grad():
    next_step_probs = planner(state_tensor)
    next_step = next_step_probs.argmax().item()
    
print(f"\nPredikovaný další krok: pozice ({next_step % 20}, {next_step // 20})")

## 6. Interaktivní A* explorátor 🎮

Vytvoříme Gradio aplikaci pro interaktivní experimentování s A*.

In [ ]:
# Interaktivní A* explorátor
class AStarExplorer:
    def __init__(self, neural_model=None):
        self.neural_model = neural_model
        self.current_world = None
        self.search_results = {}
    
    def create_world(self, size, obstacle_density):
        """Vytvoří nový svět"""
        self.current_world = GridWorld(size, size, obstacle_density)
        
        # Vizualizace
        fig, ax = plt.subplots(figsize=(8, 8))
        ax.imshow(self.current_world.grid, cmap='binary', origin='lower')
        ax.plot(0, 0, 'go', markersize=15, label='Start')
        ax.plot(size-1, size-1, 'ro', markersize=15, label='Cíl')
        ax.grid(True, alpha=0.3)
        ax.legend()
        ax.set_title(f'Mřížka {size}x{size}, hustota překážek: {obstacle_density:.1%}')
        
        plt.tight_layout()
        plt.savefig('current_world.png', dpi=150, bbox_inches='tight')
        plt.close()
        
        return 'current_world.png', "Svět vytvořen!"
    
    def run_astar_with_heuristic(self, heuristic_type, show_steps):
        """Spustí A* s vybranou heuristikou"""
        if self.current_world is None:
            return None, "Nejprve vytvořte svět!"
        
        graph = self.current_world.to_graph()
        start = (0, 0)
        goal = (self.current_world.width-1, self.current_world.height-1)
        
        # Výběr heuristiky
        if heuristic_type == "Manhattan":
            heuristic = lambda n: abs(n[0] - goal[0]) + abs(n[1] - goal[1])
        elif heuristic_type == "Euclidean":
            heuristic = lambda n: np.sqrt((n[0] - goal[0])**2 + (n[1] - goal[1])**2)
        elif heuristic_type == "Neural" and self.neural_model:
            def neural_h(n):
                grid_input = torch.zeros(1, 3, self.current_world.height, 
                                       self.current_world.width)
                grid_input[0, 0] = torch.FloatTensor(self.current_world.grid)
                grid_input[0, 1, n[1], n[0]] = 1
                grid_input[0, 2, goal[1], goal[0]] = 1
                
                with torch.no_grad():
                    pred = self.neural_model(
                        grid_input,
                        torch.FloatTensor([[n[0], n[1]]]),
                        torch.FloatTensor([[goal[0], goal[1]]])
                    )
                return pred.item()
            heuristic = neural_h
        else:
            heuristic = lambda n: 0  # Dijkstra
        
        # Spuštění A*
        astar = AStarVisualizer()
        path = astar.search(graph, start, goal, heuristic)
        
        # Vytvoření animace nebo finálního obrazu
        if show_steps:
            # Animace kroků
            fig = plt.figure(figsize=(10, 10))
            camera = Camera(fig)
            
            for i in range(0, len(astar.search_history), max(1, len(astar.search_history)//20)):
                plt.clf()
                self.current_world.visualize(path=None, 
                                           search_history=astar.search_history, 
                                           step=i)
                camera.snap()
            
            # Finální cesta
            plt.clf()
            self.current_world.visualize(path=path)
            camera.snap()
            
            animation = camera.animate(interval=200)
            animation.save('astar_animation.gif', writer='pillow')
            plt.close()
            
            output_file = 'astar_animation.gif'
        else:
            # Pouze finální výsledek
            fig = self.current_world.visualize(path=path)
            plt.savefig('astar_result.png', dpi=150, bbox_inches='tight')
            plt.close()
            output_file = 'astar_result.png'
        
        # Statistiky
        stats = f"""
        📊 Výsledky A* s {heuristic_type} heuristikou:
        - Délka cesty: {len(path) if path else 'Cesta nenalezena'}
        - Prozkoumaných uzlů: {len(astar.closed_set)}
        - Uzlů ve frontě: {len(astar.open_set)}
        - Efektivita: {len(path)/len(astar.closed_set)*100:.1f}% (cesta/prozkoumané)
        """
        
        return output_file, stats
    
    def compare_all_heuristics(self):
        """Porovná všechny heuristiky na aktuálním světě"""
        if self.current_world is None:
            return None, "Nejprve vytvořte svět!"
        
        comparator = HeuristicComparator(self.neural_model)
        results = comparator.compare_on_problem(
            self.current_world, 
            (0, 0), 
            (self.current_world.width-1, self.current_world.height-1)
        )
        
        # Vytvoření grafu porovnání
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
        
        # Graf prozkoumaných uzlů
        heuristics = list(results.keys())
        nodes_explored = [results[h]['nodes_explored'] for h in heuristics]
        
        ax1.bar(heuristics, nodes_explored, color=['red', 'blue', 'green', 'orange'])
        ax1.set_ylabel('Počet prozkoumaných uzlů')
        ax1.set_title('Efektivita heuristik')
        ax1.grid(True, alpha=0.3)
        
        # Tabulka výsledků
        ax2.axis('tight')
        ax2.axis('off')
        
        table_data = [['Heuristika', 'Délka cesty', 'Prozkoumané uzly', 'Efektivita']]
        for h in heuristics:
            path_len = results[h]['path_length']
            nodes = results[h]['nodes_explored']
            efficiency = f"{path_len/nodes*100:.1f}%" if nodes > 0 else "N/A"
            table_data.append([h.capitalize(), str(path_len), str(nodes), efficiency])
        
        table = ax2.table(cellText=table_data, loc='center', cellLoc='center')
        table.auto_set_font_size(False)
        table.set_fontsize(10)
        table.scale(1, 2)
        
        plt.tight_layout()
        plt.savefig('heuristic_comparison.png', dpi=150, bbox_inches='tight')
        plt.close()
        
        return 'heuristic_comparison.png', "Porovnání dokončeno!"

# Vytvoření Gradio rozhraní
explorer = AStarExplorer(neural_model=model)

with gr.Blocks(title="A* Explorer") as demo:
    gr.Markdown("# 🌟 Interaktivní A* Explorátor")
    
    with gr.Tab("Vytvoření světa"):
        with gr.Row():
            with gr.Column():
                size_slider = gr.Slider(
                    minimum=10, maximum=30, value=20, step=5,
                    label="Velikost mřížky"
                )
                obstacle_slider = gr.Slider(
                    minimum=0.0, maximum=0.4, value=0.15, step=0.05,
                    label="Hustota překážek"
                )
                create_btn = gr.Button("Vytvořit svět", variant="primary")
            
            with gr.Column():
                world_image = gr.Image(label="Aktuální svět")
                world_status = gr.Textbox(label="Status")
        
        create_btn.click(
            fn=explorer.create_world,
            inputs=[size_slider, obstacle_slider],
            outputs=[world_image, world_status]
        )
    
    with gr.Tab("Spuštění A*"):
        with gr.Row():
            with gr.Column():
                heuristic_select = gr.Dropdown(
                    choices=["Manhattan", "Euclidean", "Neural", "Dijkstra"],
                    value="Manhattan",
                    label="Heuristika"
                )
                show_steps_check = gr.Checkbox(
                    label="Zobrazit animaci kroků",
                    value=False
                )
                run_btn = gr.Button("Spustit A*", variant="primary")
            
            with gr.Column():
                result_image = gr.Image(label="Výsledek")
                result_stats = gr.Textbox(label="Statistiky", lines=6)
        
        run_btn.click(
            fn=explorer.run_astar_with_heuristic,
            inputs=[heuristic_select, show_steps_check],
            outputs=[result_image, result_stats]
        )
    
    with gr.Tab("Porovnání heuristik"):
        compare_btn = gr.Button("Porovnat všechny heuristiky", variant="primary")
        comparison_image = gr.Image(label="Porovnání")
        comparison_status = gr.Textbox(label="Status")
        
        compare_btn.click(
            fn=explorer.compare_all_heuristics,
            inputs=[],
            outputs=[comparison_image, comparison_status]
        )
    
    gr.Markdown(
        """
        ### 📝 Návod:
        1. **Vytvoření světa**: Nastavte velikost a hustotu překážek
        2. **Spuštění A***: Vyberte heuristiku a sledujte výsledky
        3. **Porovnání**: Porovnejte efektivitu různých heuristik
        
        ### 🧠 Heuristiky:
        - **Manhattan**: Součet absolutních rozdílů souřadnic
        - **Euclidean**: Přímá vzdálenost
        - **Neural**: Naučená heuristika pomocí neuronové sítě
        - **Dijkstra**: Nulová heuristika (garantuje nejkratší cestu)
        """
    )

# Spuštění aplikace
print("🚀 Spouštím A* Explorer...")
demo.launch(share=True)

## 7. Praktické aplikace A* 🛠️

Implementace A* pro reálné problémy.

In [ ]:
# Aplikace 1: Navigace ve městě
class CityNavigator:
    def __init__(self):
        # Vytvoření grafu města
        self.city_graph = {
            'Domov': [('Park', 5), ('Obchod', 3), ('Škola', 10)],
            'Park': [('Domov', 5), ('Kavárna', 2), ('Náměstí', 4)],
            'Obchod': [('Domov', 3), ('Náměstí', 6), ('Restaurace', 4)],
            'Škola': [('Domov', 10), ('Knihovna', 2), ('Sportoviště', 3)],
            'Kavárna': [('Park', 2), ('Náměstí', 3)],
            'Náměstí': [('Park', 4), ('Obchod', 6), ('Kavárna', 3), ('Restaurace', 2)],
            'Restaurace': [('Obchod', 4), ('Náměstí', 2)],
            'Knihovna': [('Škola', 2), ('Sportoviště', 4)],
            'Sportoviště': [('Škola', 3), ('Knihovna', 4)]
        }
        
        # Souřadnice pro vizualizaci a heuristiku
        self.coordinates = {
            'Domov': (0, 0),
            'Park': (2, 3),
            'Obchod': (-2, 1),
            'Škola': (5, -1),
            'Kavárna': (3, 5),
            'Náměstí': (1, 4),
            'Restaurace': (-1, 3),
            'Knihovna': (6, 1),
            'Sportoviště': (7, -2)
        }
    
    def heuristic(self, node, goal):
        """Vzdušná vzdálenost mezi místy"""
        x1, y1 = self.coordinates[node]
        x2, y2 = self.coordinates[goal]
        return np.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    
    def find_path(self, start, goal):
        """Najde nejkratší cestu pomocí A*"""
        astar = AStarVisualizer()
        path = astar.search(
            self.city_graph, start, goal,
            lambda n: self.heuristic(n, goal)
        )
        return path, astar
    
    def visualize_city(self, path=None):
        """Vizualizuje město a cestu"""
        G = nx.Graph()
        
        # Přidání uzlů a hran
        for node, edges in self.city_graph.items():
            for neighbor, weight in edges:
                G.add_edge(node, neighbor, weight=weight)
        
        plt.figure(figsize=(12, 8))
        
        # Pozice uzlů
        pos = self.coordinates
        
        # Kreslení grafu
        nx.draw_networkx_nodes(G, pos, node_color='lightblue', 
                              node_size=2000, alpha=0.9)
        nx.draw_networkx_labels(G, pos, font_size=10, font_weight='bold')
        
        # Hrany s váhami
        nx.draw_networkx_edges(G, pos, alpha=0.5)
        edge_labels = nx.get_edge_attributes(G, 'weight')
        nx.draw_networkx_edge_labels(G, pos, edge_labels)
        
        # Zvýraznění cesty
        if path:
            path_edges = [(path[i], path[i+1]) for i in range(len(path)-1)]
            nx.draw_networkx_edges(G, pos, edgelist=path_edges, 
                                  edge_color='red', width=4, alpha=0.8)
        
        plt.title('Navigace ve městě pomocí A*')
        plt.axis('off')
        plt.tight_layout()
        plt.show()

# Test navigace
navigator = CityNavigator()
print("🏙️ Navigace ve městě:")

# Najdi cestu
start = 'Domov'
goal = 'Kavárna'
path, astar_obj = navigator.find_path(start, goal)

if path:
    print(f"\nCesta z {start} do {goal}:")
    print(" → ".join(path))
    
    # Výpočet celkové vzdálenosti
    total_distance = 0
    for i in range(len(path)-1):
        for neighbor, weight in navigator.city_graph[path[i]]:
            if neighbor == path[i+1]:
                total_distance += weight
                break
    
    print(f"Celková vzdálenost: {total_distance}")
    print(f"Prozkoumaných míst: {len(astar_obj.closed_set)}")
    
    # Vizualizace
    navigator.visualize_city(path)

# Aplikace 2: 15-puzzle řešič
class PuzzleSolver:
    def __init__(self):
        self.size = 4
        self.goal_state = tuple(range(16))  # 0,1,2,...,15
    
    def manhattan_distance(self, state):
        """Manhattan vzdálenost všech dlaždic od cílových pozic"""
        distance = 0
        for idx, value in enumerate(state):
            if value != 0:
                current_row, current_col = idx // 4, idx % 4
                goal_row, goal_col = value // 4, value % 4
                distance += abs(current_row - goal_row) + abs(current_col - goal_col)
        return distance
    
    def get_neighbors(self, state):
        """Vrátí možné tahy"""
        state = list(state)
        empty_idx = state.index(0)
        row, col = empty_idx // 4, empty_idx % 4
        
        neighbors = []
        moves = [(-1, 0, 'UP'), (1, 0, 'DOWN'), (0, -1, 'LEFT'), (0, 1, 'RIGHT')]
        
        for dr, dc, move in moves:
            new_row, new_col = row + dr, col + dc
            if 0 <= new_row < 4 and 0 <= new_col < 4:
                new_idx = new_row * 4 + new_col
                new_state = state.copy()
                new_state[empty_idx], new_state[new_idx] = new_state[new_idx], new_state[empty_idx]
                neighbors.append((tuple(new_state), 1))
        
        return neighbors
    
    def solve(self, initial_state):
        """Řeší puzzle pomocí A*"""
        # Převod na graf formát
        def graph_func(state):
            return self.get_neighbors(state)
        
        graph = defaultdict(lambda: graph_func)
        
        # Spuštění A*
        astar = AStarVisualizer()
        path = astar.search(
            graph, initial_state, self.goal_state,
            lambda s: self.manhattan_distance(s)
        )
        
        return path

# Test puzzle řešiče
print("\n🧩 15-Puzzle řešič:")
solver = PuzzleSolver()

# Jednoduchý testovací případ (několik tahů od cíle)
test_state = (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 0, 13, 14, 15, 12)
print(f"Počáteční stav: {test_state}")
print(f"Manhattan vzdálenost: {solver.manhattan_distance(test_state)}")

# Řešení (pozor - pro složitější stavy může trvat dlouho!)
# path = solver.solve(test_state)
# if path:
#     print(f"Řešení nalezeno! Počet tahů: {len(path)-1}")

## 8. Praktická cvičení 📝

In [ ]:
# Cvičení 1: Implementujte vlastní heuristiku
def my_custom_heuristic(node, goal, world):
    """
    TODO: Vytvořte vlastní heuristiku, která kombinuje:
    - Manhattan vzdálenost
    - Počet překážek v přímé linii k cíli
    - Preferenci určitých směrů
    """
    # Váš kód zde
    pass

# Cvičení 2: Weighted A*
class WeightedAStar:
    """
    TODO: Implementujte Weighted A*, který používá:
    f(n) = g(n) + w * h(n)
    kde w > 1 urychluje hledání za cenu suboptimality
    """
    def __init__(self, weight=1.5):
        self.weight = weight
        # Váš kód zde
    
    def search(self, graph, start, goal, heuristic):
        # Váš kód zde
        pass

# Cvičení 3: Neuronová síť s attention mechanismem
class AttentionHeuristic(nn.Module):
    """
    TODO: Vytvořte neuronovou heuristiku s attention mechanismem,
    která se "dívá" na důležité části mapy
    """
    def __init__(self):
        super(AttentionHeuristic, self).__init__()
        # Váš kód zde
        pass
    
    def forward(self, grid, current, goal):
        # Váš kód zde
        pass

# Cvičení 4: Multi-heuristický A*
def multi_heuristic_astar(graph, start, goal, heuristics, weights):
    """
    TODO: Implementujte A*, který kombinuje více heuristik
    s různými váhami: h(n) = Σ(wi * hi(n))
    """
    # Váš kód zde
    pass

print("📚 Cvičení připravena!")
print("\nTipy:")
print("- Pro cvičení 1: Použijte Bresenhamův algoritmus pro linii")
print("- Pro cvičení 2: Upravte pouze výpočet f-score")
print("- Pro cvičení 3: Inspirujte se transformer architekturou")
print("- Pro cvičení 4: Normalizujte heuristiky před kombinací")

## 9. Shrnutí a klíčové koncepty 🎓

### Co jsme se naučili:

1. **A* Algoritmus**
   - Kombinuje g(n) - skutečnou cestu a h(n) - odhad do cíle
   - f(n) = g(n) + h(n)
   - Garantuje optimální řešení pokud je h(n) admissible

2. **Heuristiky**
   - Manhattan: |x1-x2| + |y1-y2|
   - Euklidovská: √((x1-x2)² + (y1-y2)²)
   - Neuronové: učené z dat

3. **Neuronové přístupy**
   - CNN pro extrakci příznaků z mřížek
   - Transformery pro plánování sekvencí
   - Učení heuristik z optimálních řešení

4. **Praktické aplikace**
   - Navigace v mapách
   - Řešení puzzle
   - Plánování cest v robotice

### Vlastnosti dobré heuristiky:
- **Admissible**: Nikdy nepřeceňuje skutečnou vzdálenost
- **Consistent**: h(n) ≤ c(n,n') + h(n')
- **Informative**: Čím blíže skutečné vzdálenosti, tím lépe

### Další kroky:
- V další hodině: **CSP - Constraint Satisfaction Problems**
- Naučíme se řešit problémy s omezeními
- Použijeme neuronové sítě pro CSP solving

In [ ]:
# Závěrečná ukázka
print("🎉 Gratulujeme! Dokončili jste hodinu o informovaném prohledávání!")
print("\n📊 Vaše pokroky:")
print("✅ Implementace A* algoritmu")
print("✅ Vytvoření neuronových heuristik")
print("✅ Porovnání různých přístupů")
print("✅ Transformer architektury pro plánování")
print("✅ Praktické aplikace na reálné problémy")

# Shrnutí algoritmů
print("\n📋 Srovnání prohledávacích algoritmů:")
comparison = {
    'Algoritmus': ['BFS', 'DFS', 'Dijkstra', 'A*'],
    'Informovaný': ['Ne', 'Ne', 'Ne', 'Ano'],
    'Optimální': ['Ano*', 'Ne', 'Ano', 'Ano**'],
    'Složitost': ['O(b^d)', 'O(b^m)', 'O(E log V)', 'O(b^d)']
}

import pandas as pd
df = pd.DataFrame(comparison)
print(df.to_string(index=False))
print("\n* Pro neohodnocené grafy")
print("** Pokud je heuristika admissible")

print("\n🚀 Připraveni na CSP v další hodině!")

### Hodina 13 — Prohledávání do hloubky (DFS) — ELI10

DFS (Depth‑First Search) je jako když v bludišti jdeme stále rovně dál, dokud nenarazíme na slepou uličku, pak se vrátíme a zkusíme další cestu. Dobré pro prohledání všech cest a pro úkoly jako topologické řazení nebo detekce cyklů.

In [ ]:
# DFS iterative example using stack

def dfs_iterative(graph, start):
    stack = [start]
    visited = set()
    order = []
    while stack:
        node = stack.pop()
        if node in visited:
            continue
        visited.add(node)
        order.append(node)
        # push neighbors in reverse for deterministic order
        for nbr in reversed(graph.get(node, [])):
            if nbr not in visited:
                stack.append(nbr)
    return order

# Test graph
G = {
    'A': ['B','C'],
    'B': ['D','E'],
    'C': ['F'],
    'D': [],
    'E': [],
    'F': []
}
order = dfs_iterative(G, 'A')
print('DFS order:', order)
assert isinstance(order, list)
assert order[0] == 'A'
print('DFS checks passed')

Úkoly:

1) Implementujte rekurzivní verzi DFS a porovnejte pořadí s iterativní verzí.
2) Použijte DFS k detekci cyklů v grafu (oriented graph) a vytvořte jednoduchý test.
3) Bonus: Implementujte topologické řazení pro acyklický graf (DAG).